# Booking hotel crawler

In [16]:
# import lib
from datetime import datetime, timedelta, date
import requests
from bs4 import BeautifulSoup
import re
import time

最多查詢到明年年底

In [2]:
# Settings
surveyRange = 5
travelPeriod = 4

In [3]:
# Date List 
# args: start date, survey range, travel period(days)
# ex. [datetime.date(2017, 8, 7), datetime.date(2017, 8, 11)] ~ [datetime.date(2017, 9, 5), datetime.date(2017, 9, 9)]]
def getDateList(dt = date.today(), ran = surveyRange, period = travelPeriod):
    timeList = []
    start = dt
    end = start + timedelta(days = period)
    for i in range(1, ran+1):  # i = 1 ~ ran
        timeList.append([start, end])
        start = start + timedelta(days = 1)
        end = end + timedelta(days = 1) 
    return timeList
timeList = getDateList()

In [4]:
# Date List components
checkinM = []
checkinD = []
checkinY = []
checkoutM = []
checkoutD = []
checkoutY = []
for i in timeList:
    checkinM.append(i[0].month)
    checkinD.append(i[0].day)
    checkinY.append(i[0].year)
    checkoutM.append(i[1].month)
    checkoutD.append(i[1].day)
    checkoutY.append(i[1].year)

In [5]:
# headers
# cookies
# location
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
cookie = {"Cookie":"header_signin_prompt=1; zz_cook_tms_seg1=1; zz_cook_tms_seg3=7; zz_cook_tms_ed=1; has_preloaded=1; _ga=GA1.2.2061587162.1502107968; _gid=GA1.2.1814322313.1502107968; BJS=-; utag_main=v_id:015dbc9ca2c900457102ac7642380407300bf06b00978$_sn:1$_ss:0$_st:1502109920031$ses_id:1502107968202%3Bexp-session$_pn:3%3Bexp-session$4split:0$4split2:0; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbKE7bjkbYWzncUcat06CAsbQ%2B9%2FAkzVENB8nmBa8Vs%2F%2BVVB4vsfV9CjFR48ycz7Yx9EStGOdtWO3JGmN9AgMB0%2BVUgMcxBgoCrfKMX2yn9L%2FD3c0q2Li1pkMyK7ILqx8bL20GYH8glGAe%2FhsFvdJwERKlgvjQM1TPezbzeLBGAiQ%3D; lastSeen=0; _gat=1"}
location = '京都'

In [6]:
# Pages amount for each travel period.  
# ex. numHotelList = [ ['145'], ['160'], ['167'], ['203'], ['239'], ['294'], ['353'], ['371'], ... increasing by date to date

pagesList = []
numHotelList = []
turnPageList = []
for i in range(1, surveyRange+1): # i = 1 ~ surveyRange
    # url位址為XHR文件中的searchresults，去除AJAX部分
    url = "https://www.booking.com/searchresults.zh-tw.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=25f3195cbf76726e8ffac861269a0d24&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ssb=empty&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50"###+"&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdPHUBSSUVJfcbWYaNLDRCAET&_=1501050897864"
    res = requests.get(url, headers = headers, cookies = cookie)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 找總頁數
    clue1 = soup.select('#right')[0].find(lambda x: x.get('data-et-view') == 'TULQWNVLBLWIJDAOfUYdXaO:1')
    pattern = "找到\s(\d+)\s間" #找頁數用
    numHotelText = re.findall(pattern, clue1.find('h1').text)
    # 這區間共幾間hotel
    numHotel = int(numHotelText[0])
    # numHotel / 一頁有50筆  =  翻頁數
    turnPage = round(numHotel/50)
    # append to lists
    pagesList.append(numHotel)
    turnPageList.append(turnPage)

In [17]:
# for each period (ex. 10/7 ~ 10/11), grab the hotelList information

for i in range(1, surveyRange+1): # i = 1 ~ surveyRange
    url = "https://www.booking.com/searchresults.zh-tw.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=25f3195cbf76726e8ffac861269a0d24&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ssb=empty&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50"###+"&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdPHUBSSUVJfcbWYaNLDRCAET&_=1501050897864"
    res = requests.get(url, headers = headers, cookies = cookie)
    soup = BeautifulSoup(res.text, 'lxml')
    
    for turnPage in turnPageList: # turnPageList: ex. [3, 3, 3, 3, 4]
        for page in range(1, turnPage+1): # page = 1 ~ turnPage
            offset = (page-1)*50  # offset: from index of hotel(in below url)
            url2 = "https://www.booking.com/searchresults.zh-tw.html?label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmaOcBiAEBmAEwuAEGyAEM2AED6AEBkgIBeagCAw&sid=4996667e149bfae490c17f2eb37eafa9&checkin_month="+str(checkinM[i-1])+"&checkin_monthday="+str(checkinD[i-1])+"&checkin_year="+str(checkinY[i-1])+"&checkout_month="+str(checkoutM[i-1])+"&checkout_monthday="+str(checkoutD[i-1])+"&checkout_year="+str(checkoutY[i-1])+"&class_interval=1&dest_id=-235402&dest_type=city&dtdisc=0&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss="+str(location)+"&ss_all=0&ssb=empty&sshis=0&ssne="+str(location)+"&ssne_untouched="+str(location)+"&rows=50&offset="+ str(offset)### + "&sr_ajax=1&b_gtt=dLYAeZFVJfNTBdAHUdHPGZDfIfVNASUcbTYWPJLO&_=1501678454525"        
            res2 = requests.get(url2, headers = headers, cookies = cookie)
            soup2 = BeautifulSoup(res2.text, 'lxml')
            items = soup2.select(".sr_item")
            with open('BookingCrawl_now-{}-{}-{}-from-{}-to-{}-Page{}.txt'.format(time.localtime().tm_mon, time.localtime().tm_mday, time.localtime().tm_hour, timeList[i-1][0], timeList[i-1][1], page), 'w', encoding = 'UTF-8') as f:
                f.write(str(items))

IndexError: list index out of range

In [19]:
timeList

[[datetime.date(2017, 8, 8), datetime.date(2017, 8, 12)],
 [datetime.date(2017, 8, 9), datetime.date(2017, 8, 13)],
 [datetime.date(2017, 8, 10), datetime.date(2017, 8, 14)],
 [datetime.date(2017, 8, 11), datetime.date(2017, 8, 15)],
 [datetime.date(2017, 8, 12), datetime.date(2017, 8, 16)]]

In [18]:
turnPageList

[3, 3, 3, 3, 4]

In [20]:
surveyRange

5